In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory
from tqdm.notebook import tqdm
from itertools import product
from matplotlib.patches import Rectangle
import pandas as pd

from pyomo_models.task_allocation import TaskAllocationModel, Scenario, Parameters, Schedule

In [ ]:
sc = Scenario.from_file("scenarios/task_allocation/students.yaml")

In [ ]:
N_t = 5
v = np.array([1, 1, 1])
r_charge = np.array([0.02, 0.02, 0.02])
r_deplete_fly = np.array([0.00125, 0.00125, 0.00125])
# r_deplete_fly = np.array([0.0025, 0.0025, 0.0025])
r_deplete_inspect = np.array([0.01, 0.01, 0.01])
i = np.array([1, 1, 2])
B_start = [1, 1, 1]
B_min = 0
B_max = 1
R = np.array([30, 30, 30, 30, 30, 30, 60, 60, 60])
parameters = {p: eval(p) for p in ['N_t', 'v', 'r_charge', 'r_deplete_fly', 'r_deplete_inspect', 'i', 'B_start', 'B_min', 'B_max', 'R']}

In [ ]:
model = TaskAllocationModel(sc, parameters)

In [ ]:
solver = SolverFactory('gurobi')
solver.options['TimeLimit'] = 120
# solver.options['MIPFocus'] = 0
# solver.options['Cuts'] = -1
solution = solver.solve(model, tee=True) 

In [ ]:
print(solution)

In [ ]:
for d in model.d:
    i = model.inspection_time(d)()
    c = model.charging_time(d)()
    total = i + c
    print(f"Execution time for '{d}': {i:.2f} + {c:.2f} = {total:.2f}s")

In [ ]:
schedule = Schedule(model)

### Path

In [ ]:
fig, axes = plt.subplots(1, model.N_d, figsize=(3*model.N_d, 5), sharex=True, sharey=True)
fig.subplots_adjust(hspace=.4)
for d in model.d:
    sc.plot(ax=axes[d])
    schedule.plot_path(d, sc, ax=axes[d], zorder=-1)
    axes[d].set_title(f"Drone ${d}$")
    # axes[d].axis("off")
plt.savefig("out/task_allocation/paths.pdf", bbox_inches='tight')
lines, labels = axes[0].get_legend_handles_labels() 
fig.legend(lines, labels, bbox_to_anchor=(0.5, 0), ncol=3, loc='center')
plt.show()

### Battery profile

The figure below shows the battery life of the drone over time. 
The red parts correspond to the inspection period.

In [ ]:
fig, axes = plt.subplots(model.N_d, 1, figsize=(5, 1.5*model.N_d), sharex=True)
fig.subplots_adjust(hspace=.6)
for d in model.d:
    schedule.plot_battery(d, ax=axes[d])
    axes[d].set_title(f"$d={d}$")
axes[1].set_ylabel("battery charge")
plt.xlabel("time")
plt.savefig("out/task_allocation/battery_profile.pdf", bbox_inches='tight')
plt.show()

### Inspection effort

The table below illustrates the inspection 'effort' performed by each drone for each waypoint.

In [ ]:
data = []
for w in model.w:
    row = (schedule.P[:,w,:] * schedule.I * model.i.reshape((model.i.shape[0], 1))).sum(axis=1)
    data.append(row)
df = pd.DataFrame(data, columns=[f"d{d}" for d in model.d])
df.index = [f"wp{wp}" for wp in df.index]
df['sum'] = df.sum(axis=1)

In [ ]:
df

# Verify battery depletion

#### Flying

In [ ]:
y = np.reshape(model.y[:,:,:,:](), (model.N_d, model.N_n, model.N_n, model.N_t-1))
idx = np.argwhere(y == 1)

distance_traveled = {d:[sum(model.D_start[d] * model.P[d,:,0]())] for d in model.d} # start distance

for d, n, n_prime, t_r in idx:
    distance = model.D[n, n_prime]
    distance_traveled[d].append(distance)

In [ ]:
for k,v in distance_traveled.items():
    print("{}: {} sums to {:.1f}".format(k, [f"{x:.1f}" for x in v], sum(v)))

These distances can be turned into battery depletion:

In [ ]:
for k,v in distance_traveled.items():
    print(f"{k}: {sum(v)/8:.1f}%")

#### Inspection

In [ ]:
for d in model.d:
    depleted = (schedule.I[d,:] * model.r_deplete_inspect[d]).sum()*100
    print(f"{d}: {depleted:.2f}%")

#### Total

In [ ]:
for d in model.d:
    print(f"{d}: {sum(distance_traveled[d])/8 + (schedule.I[d,:] * model.r_deplete_inspect[d]).sum()*100:.1f}%")

# Output of the model

Decision matrix (which nodes to follow). The matrix denotes whether a drone moves to node $n$ (vertical axis) at time step $t$ (horizontal axis).

In [ ]:
for d in model.d:
    print(f"Drone {d}:")
    print(schedule.P[d])
    print()

Decision matrix of the inspection duration at a time step $t$

In [ ]:
for d in model.d:
    print(f"Drone {d}:")
    print(schedule.I[d].round(2))
    print()